---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [37]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split

def blight_model():
    train_df = pd.read_csv("train.csv", encoding = 'ISO-8859-1')
    train_df = train_df[(train_df["compliance"] == 0) | (train_df["compliance"] == 1)]
    
    #addresses_df = pd.read_csv("addresses.csv")
    #latlon_df = pd.read_csv("latlons.csv")
    
    y_train_df = train_df[["compliance"]]
    
    #filtering down to relevant features
    features_df = train_df[["state", "disposition",
                            "discount_amount", "clean_up_cost", "judgment_amount", "violation_code"]]
    #here's the test data
    test_df = pd.read_csv("test.csv", encoding = 'ISO-8859-1')
    
    #null values
    test_df["state"][pd.isnull(test_df["state"])] = "Nan"
    #test_df["city"][pd.isnull(test_df["city"])] = "Nan"
    
    #transform states into labels
    state_le=LabelEncoder()
    features_df["state"][pd.isnull(features_df["state"])] = "Nan"
    
    all_states = pd.concat((pd.Series(features_df["state"]), pd.Series(test_df["state"]))).unique()
    state_le.fit(all_states)
    features_df["state"] = state_le.transform(features_df["state"])
    test_df["state"] = state_le.transform(test_df["state"])
    
    #transform disposition into labels
    dispos_le = LabelEncoder()
    all_dispositions = pd.concat((pd.Series(features_df["disposition"]), pd.Series(test_df["disposition"]))).unique()
    dispos_le.fit(all_dispositions)
    features_df["disposition"] = dispos_le.transform(features_df["disposition"])
    test_df["disposition"]= dispos_le.transform(test_df["disposition"])
    
    #transform violation_code into labels
    viocode_le = LabelEncoder()
    all_viocode = pd.concat((pd.Series(features_df["violation_code"]), pd.Series(test_df["violation_code"]))).unique()
    viocode_le.fit(all_viocode)
    features_df["violation_code"] = viocode_le.transform(features_df["violation_code"])
    test_df["violation_code"] = viocode_le.transform(test_df["violation_code"])
    
    

    test_features_df = test_df[["ticket_id", "state", "disposition",
                            "discount_amount", "clean_up_cost", "judgment_amount", "violation_code"]]
    #print(features_df.head())
    #print("test_features: ", test_features_df.head())
    test_features_df = test_features_df.set_index("ticket_id")
    
    clf = RandomForestClassifier().fit(features_df, y_train_df)

    y_score = clf.predict_proba(test_features_df)[:,1]

    final_df = pd.DataFrame(y_score, index = test_features_df.index)
    
    
    """
    X_train, X_test, y_train, y_test= train_test_split(features_df, y_train_df, random_state = 0)
    y_score = clf.predict(X_train)
    print('train score ', roc_auc_score(y_train, y_score))
    
    y_test_score = clf.predict(X_test)
    print('test score ', roc_auc_score(y_test, y_test_score))
    
    print("trainscore_ ", clf.score(X_train, y_train))
    print("test score_ ", clf.score(X_test, y_test))
    """
    return final_df

blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/pandas/core/generic.py:4702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/

,0
ticket_id,
284932,0.157097
285362,0.010698
285361,0.052183
285338,0.043333
285346,0.075347
285345,0.043333
285347,0.061966
285342,0.866755
285530,0.000000


In [7]:
import pandas as pd 
train_df = pd.read_csv("train.csv", encoding = 'ISO-8859-1')
test_df = pd.read_csv("test.csv", encoding = 'ISO-8859-1')

train_df = train_df.set_index(train_df["ticket_id"])
print(train_df.head())

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


           ticket_id                                     agency_name  \
ticket_id                                                              
22056          22056  Buildings, Safety Engineering & Env Department   
27586          27586  Buildings, Safety Engineering & Env Department   
22062          22062  Buildings, Safety Engineering & Env Department   
22084          22084  Buildings, Safety Engineering & Env Department   
22093          22093  Buildings, Safety Engineering & Env Department   

             inspector_name                      violator_name  \
ticket_id                                                        
22056       Sims, Martinzie  INVESTMENT INC., MIDWEST MORTGAGE   
27586      Williams, Darrin           Michigan, Covenant House   
22062       Sims, Martinzie                    SANDERS, DERRON   
22084       Sims, Martinzie                       MOROSI, MIKE   
22093       Sims, Martinzie                    NATHANIEL, NEAL   

           violation_street_numb

In [8]:
df = train_df.groupby("violation_code").compliance.agg(['sum','count']).reset_index()
df["percentage"] = df['sum']/df['count']
df

,violation_code,sum,count,percentage
0,19410901,NaN,0,NaN
1,19420901,4.0,16,0.250000
2,19450901,45.0,1620,0.027778
3,19830901,0.0,10,0.000000
4,19840901,1.0,2,0.500000
5,19850901,0.0,1,0.000000
6,20130901,3.0,78,0.038462
7,20160901,0.0,1,0.000000
8,20180901,2.0,15,0.133333
9,22-2-16,0.0,17,0.000000
